In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import open3d as o3d

# Data Preprocessing

In [ ]:
# Read file 
cloud_df = pd.read_csv('cloud.txt', header=None, sep=' ')
xyz = cloud_df[[0,1,2]].values
intensities = cloud_df[3].values

In [ ]:
# Build point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

pcd_center = pcd.get_center()
pcd.translate(-pcd_center)
pcd.colors = o3d.utility.Vector3dVector(np.tile(intensities[:, np.newaxis], (1, 3)))  # Assign intensities as colors


In [ ]:
# Setting the view
view = {
  "front" : [ -0.97399899973224224, -0.18506685251446897, 0.13067596803153753 ],
  "lookat" : [ 37.586813403376482, 5.9613108866587297, -4.6193071077826815 ],
  "up" : [ -0.0015264409403050001, 0.01464551045165827, 0.99989158362377784 ],
  "zoom" : 0.45230139160156235
}

In [ ]:
# Visualize
o3d.visualization.draw_geometries([pcd], **view)

## Downsample

- Can reduce computation and noise level.  
- However, this data does not benefit too much from voxel downsampling.

In [ ]:
# Voxel Downsampling
pcd_down = pcd.voxel_down_sample(voxel_size=0.05)

In [ ]:
o3d.visualization.draw_geometries([pcd_down], **view)

## Noise removal

Statistical filter to remove outlier points

In [ ]:
# Noise filtering with nearest neighbours
nn = 16 # NN to consider

std_multiplier = 8 # Remove if greater than this std multiplier 

filtered_pcd, filtered_idx = pcd_down.remove_statistical_outlier(nn, std_multiplier)
outliers = pcd_down.select_by_index(filtered_idx, invert = True)

outliers.paint_uniform_color([1,0,0]) # Color outliers


In [ ]:
o3d.visualization.draw_geometries([filtered_pcd, outliers], **view) 

# Begin clustering

In [ ]:
# Start clustering
labels = np.array(filtered_pcd.cluster_dbscan(eps=0.8, min_points = 10))
max_label = labels.max()

colors = plt.get_cmap('tab20') (labels / (max_label) if max_label>0 else 1)
colors[labels<0] = 0 # Label outliers as black

filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3]) # Assign color to each cluster

In [ ]:
o3d.visualization.draw_geometries([filtered_pcd], **view)

## Asign bounding box for each cluster

In [ ]:
# Make a bounding box for each cluster
obs = []
label_range = range(0, labels.max()+1)
# Iterate over clusters and find bounding box
for l in label_range:
    points = filtered_pcd.select_by_index(np.squeeze(np.argwhere(labels==l)))
    if sum(labels==l):
        obb = points.get_axis_aligned_bounding_box()
        obb.color = points.colors[0]
        obs.append(obb)

In [ ]:
o3d.visualization.draw_geometries([filtered_pcd] + obs, **view)